# Setting up your Web Server


## I have done all of this for you!  This is only for your information (if you want to do it yourself on your own computer)

The Web Server we are using is called "Lighttpd"

To install this:

    $ sudo apt-get install lighttpd

To configure it for dynamic pages, generated by Ruby scripts, you need to do the following configuration steps:

    $ sudo lighty-enable-mod cgi

    $ sudo pico /etc/lighttpd/lighttpd.conf

        # enable "mod_setenv"
        server.modules = (
            "mod_access",
            "mod_alias",
            "mod_compress",
            "mod_redirect",
            "mod_setenv"     # need to switch this on for Ruby
        )

        server.document-root        = "/var/www/html"
        server.upload-dirs          = ( "/var/cache/lighttpd/uploads" )
        server.errorlog             = "/var/log/lighttpd/error.log"
        server.breakagelog          = "/var/log/lighttpd/breakage.log"                #  This is for CGI error tracking!
        server.pid-file             = "/var/run/lighttpd.pid"
        server.username             = "www-data"
        server.groupname            = "www-data"
        server.port                 = 80


        index-file.names            = ( "index.php", "index.html", "index.lighttpd.html" )
        url.access-deny             = ( "~", ".inc" )
        static-file.exclude-extensions = ( ".php", ".pl", ".fcgi", ".rb" )    # tell the server .rb is NOT a static file

        compress.cache-dir          = "/var/cache/lighttpd/compress/"
        compress.filetype           = ( "application/javascript", "text/css", "text/html", "text/plain" )

        # default listening port for IPv6 falls back to the IPv4 port
        ## Use ipv6 if available
        #include_shell "/usr/share/lighttpd/use-ipv6.pl " + server.port
        include_shell "/usr/share/lighttpd/create-mime.assign.pl"
        include_shell "/usr/share/lighttpd/include-conf-enabled.pl"





    $ sudo pico /etc/lighttpd/conf-enabled/10-cgi.conf  

    ....look for the line $HTTP['url'] and edit it as follows:
    

            $HTTP["url"] =~ "^/interfaces/" {
                                            cgi.assign = ( "" => "/usr/bin/ruby" ),
                                                 setenv.add-environment = (             # this is why you need mod_setenv
          "GEM_PATH" => "/home/osboxes/.rvm/gems/ruby-2.4.2:/home/osboxes/.rvm/gems/ruby-2.4.2@global"
                )

            }


Then reload the server:

    $ sudo /etc/init.d/lighttpd force-reload 


    $ sudo chown -R osboxes.osboxes /var/www/html
         (whatever your username is… on this machine it is 'osboxes')
         
    $ cd /var/www/html
    
    $ mkdir interfaces
    
    $ cd interfaces
    
    $ echo 'print "Content-Type:text/html\n\nhello";' > test.rb
    
    $ chmod a+x test.pl
    
<pre>


</pre>



You should now be able to surf to your own web page:  http://localhost/

You should also be able to execute that little Ruby script:  http://localhost/interfaces/test.rb


# Understanding your Web server

## Where is everything?  What does it all mean?  How does it work?

### Server Root

    /var/www/html   

This is the “root” of your Webserver.  When you surf to:  http://localhost this is the folder that the browser is accessing. if you surf to:  http://localhost/somefolder   then you are accessing  /var/www/html/somefolder on your computer

If a browser requests a folder (like http://localhost) your Web server will, by default, attempt to serve a document called "index.html" from inside of that folder.  (you can change this default behavior by modifying parts of the configuration file at <code>/etc/lighttpd/lighttpd.conf</code> as we did above).  So when you see:

        UPM Bioinformatics Programming Challenges
        Main Web Directory
       
that is the content of the file called "index.html" in your Server's root folder (go ahead, look at it now).  So, calling <code>http://localhost</code> and calling <code>http://localhost/index.html</code> give you exactly the same result.


### Script Root

    /var/www/html/interfaces/
    
In your <code>/etc/lighttpd/conf-enabled/10-cgi.conf</code> folder, you told the Server that any URL that matches "/interfaces/" should be treated as a request for a Ruby script, and it should attempt to execute that script using /usr/bin/ruby.  So, this is where you will put all of your Web-accessible Ruby scripts.  The browser will run the script, and send the OUTPUT of that script to the Web server, which then sends it to the user's Web Browser.



### Error messages

    /var/log/lighttpd/error.log
    /var/log/lighttpd/breakage.log
    
Sometimes (often!) your script will not send any output... The only thing you see is a blank browser page!  When this happens, it usually means that the script had an error.  To see what the error was, you need to look at one or the other of these two files (you will need <code>sudo</code> to view it).  Generally, <code>error.log</code> will show global errors related to, for example, mis-configuration of the server.  <code>breakage.log</code> will show you specific errors that come out of your Ruby script.  NOTE:  YOU CAN PURPOSELY SEND ERRORS TO THIS LOGFILE using STDERR!!

     $stderr.puts "In a Ruby CGI, this message will go to breakage.log... good for debugging!"
    
    


### More configuration

    $ ls /etc/lighttpd/
    conf-available  conf-enabled  lighttpd.conf

   
<code>lighttpd.conf</code> is your main server configuration file.

The <code>conf-available</code> folder contains the functionalities you can activate - things like Flash video streaming, etc.

The <code>conf-enabled</code> folder contains the functionalities you have activated.  in our case, we have activated 10-cgi.conf



<pre>


</pre>

# Static vs. Dynamic Web Content

### Static
Think of this like an MS Word document.  You open it, read it, and close it.  It is always the same unless you edit it.  It exists as a file in a folder.


### Dynamic
Think of this like an Excel Macro.  What comes out of it will depend on what goes into it.  What comes out LOOKS like a document, but it doesn’t “exist” anywhere.  It is generated when the macro is run.

We will create both kinds of Web content in the next few weeks, and your final assignment for the course will require you to create both static pages (a query form for a database) and dynamic pages (the output from the query, presented in a visually beautiful way for your users!)




<pre>
  

</pre>
  
# HTML - the (main) syntax for Web content

Hypertext Markup Language (HTML) is the language used to create most Web content (both static and dynamic!)  HTML is just a text document that contains HTML “tags”.  “tags” are formatting instructions, including things like “bold” “italic” “Heading_1” “Heading_2”, etc.

You edit static HTML documents in exactly the same way as you edit all other text documents - you can use Komodo for this (or any text editor, but it is easier to use one that "understands" HTML, like Komodo)


<pre>
  

</pre>
  
# Common Gateway Interface - CGI - for Dynamic Web content

This is the **grandfather** of all Dynamic content on the Web (I wrote my first CGI Web page when I was a student!)  There are modern alternatives to CGI that are _more efficient, more secure, often easier to maintain, and that enforce good programming habits._  The most common framework for Ruby is called "Rails" ([explained in Spanish](http://rubyonrails.org.es/).  **IF** you become bioinformaticians in the future, I would **strongly suggest that you learn one of these Web content frameworks!!**   I am teaching you CGI _only_ because it is extremely simple, and we don't need to spend an additional class talking about good Web software architecture.  I am NOT teaching you CGI because it is the "best way", or even a "good way"!  (<span style="color: red;">You have been warned!</span>) 

Effectively, CGI defines the way that a Web server will collect information from a Web browser (e.g. when you press a “submit” button on the Web) and provide that information to your Ruby script.


<pre>
  

</pre>
  
# Basic HTML - creating Static Web pages

As described earlier, HTML is the syntax for describing the layout and structure of a document in the form of "tags".  

Web browsers (almost) universally represent those "tags" in exactly the same way, so most Web pages look exactly the same  on Safari as they do on Internet Explorer or Firefox or Chrome or Opera.  However, they don't _have to_ look the same!  For example, for people with bad eyesight, or colorblindness, they can modify how their Browser represents these "tags" to make the content of the page easier for them to read.  

This is, in fact, the *purpose* of HTML - to _describe_ the structure of the information, but separate that from explicit instructions about _how to represent_ that structured information.  (Which is one of the reasons that the Web is so much better than PDF or DOCX :-) )


**NOTE:  The Markdown cells in Jupyter respond to most (all?) HTML commands, so you will sometimes need to look at the code for the cell to see the tags in the following lesson...**




## an example of HTML


    <html>
    <head>
        <title>This is my first web page</title>
    </head>
    <body>
    
    <h1>My First Web Page</h1>
    HTML programming is <b><i>so much fun</i></b>
    <br/>
    
    </body>
    <html>	
    
is "rendered" (the process of converting formatting instructions into a visual representation) like this:

------------------

<html>
<head>
	<title>This is my first web page</title>
</head>
<body>
	<h1>My First Web Page</h1>
Perl programming is <b><i>so much fun</i></b>
    <br/>
    
</body>
<html>	

------------------------

<pre>


</pre>
The structure of an HTML document follows the same rules as the structure of an XML document.  Tags are opened by putting the tag in angle-brackets, and closed by preceding the tag name with a '/'

The general structure of an HTML document is:

Starts with the tag HTML
    contains a HEAD
        contains Metadata, such as TITLE
    HEAD ENDS
    contains a Body
        contains information to be rendered
    BODY ENDS
HTML ENDS

You can see the "nested" structure of the tags here:

<a href="https://ibb.co/cda5Um"><img src="https://preview.ibb.co/mdhVvR/Selection_125.png" alt="Selection_125" border="0"></a>


Using Komodo, create a document with that structure and save it in your Web server's root folder as "fun.html", then click on this link:  http://localhost/fun.html

There you are !  Your first Web page!




<pre>
  

</pre>
  
# More HTML

Most of the interesting stuff happens in the BODY, so that's where we will focus the lecture today.

The most common HTML tags are:

    <h1> Heading 1  </h1>
    <h2> Heading 2   </h2>
    <h3>.... and so on  first, second and third-level headings  </h3>

    <b>   bold </b>
    <i>   italic </i>

---------------------

<h1> Heading 1  </h1>
<h2> Heading 2   </h2>
<h3>.... and so on  first, second and third-level headings  </h3>

<b>   bold </b>
<i>   italic </i>


-------------------
The way to make lists is:

    <ul>
        <li>the first thing</li>
        <li>the second thing</li>
    </ul>

-----------------------

<ul>
    <li>the first thing</li>
    <li>the second thing</li>
</ul>

----------------

OR

    <ol>
        <li>the first thing</li>
        <li>the second thing</li>
    </ol>
    
--------------

<ol>
    <li>the first thing</li>
    <li>the second thing</li>
</ol>

------------

then there is the general text layout instructions:

    Some text here <br/> with a newline (note that it opens and closes!)
    <p>some paragraph</p>
    <p>Another paragraph</p>

---------------

Some text here <br/> with a newline (note that it opens and closes!)
<p>some paragraph</p>
<p>Another paragraph</p>


-------------
    
everything else is just displayed as plain text.

__Note that SPACES ARE IGNORED!!__

    some text                that              looks
                         like                           this    has         no
                                       spaces
                         
----------------

some text                that              looks
                     like                           this    has         no
                                   spaces
                         
-----------------


<pre>


</pre>

# Links and Images

## Links

A "Hyperlink" is a piece of text that is assigned to a URL.  When the user clicks the text in their browser, you are sent to that URL.  The way to represent that in HTML is:


    This is a link to <a href="http://www.cbgp.upm.es">The CBGP Website</a>

------------

This is a link to <a href="http://www.cbgp.upm.es">The CBGP Website</a>

-------------

## Images

Images are also simple to use - you simply refer to the URL of the image, and use the "img" tag:


    <img src="https://c1.staticflickr.com/9/8115/8673546462_73b2df8cb3_b.jpg"/>

---------------

<img src="https://c1.staticflickr.com/9/8115/8673546462_73b2df8cb3_b.jpg" width='200px'/>

----------------

### For this course **you do not need to know** ANY more HTML than this!

If you want to learn more on your own, please do!  If you want your pages to be "fancy", then look for tutorials on the topic of "Cascading Stylesheets" (CSS) - they will tell you all of the formatting possibilities, and how to make pages that can more easily be changed to meet a new "theme" (for example, if your project decides to change its color theme, you don't have to re-write all of the HTML pages!)



<pre>
  

</pre>
  
# Dynamic Web Pages

It is surprisingly easy to create dynamic Web pages!  All you must do is write a Ruby script that outputs HTML!  (it really is that easy!!)

There’s only ONE little extra thing you must do...

### Content-type

The [HTTP Protocol Specification](https://tools.ietf.org/html/rfc7230) requires that a Web server **MUST** provide information about what “kind of stuff” it is going to send to the Browser.

When you are browsing static pages, your Web server generally decides this automatically (usually by looking at the file extension (e.g. index.<span style="color: red;">html</span>,   mypage.<span style="color: red;">txt</span>,  document.<span style="color: red;">pdf</span>)

When you are creating dynamic content from a script, you must tell the Server what kind of data you are going to send - it has no way to know!  Maybe your script produces HTML... maybe it produces XML... maybe it produces Linked Data... maybe it produces a PDF, or an Image...??  You must tell the server what the content of the HTTP message will be


In the most simplistic case, it is this:

    Content-Type: text/html
    
    body starts here......
    
So, a field named "Content-Type:"  with a value that must be [one of the types on this list](https://www.sitepoint.com/mime-types-complete-list/)

Followed by a blank line

...and that's all!

To achieve this in Ruby, you simply say:

    print "Content-Type: text/html  \n\n";



<pre>
  

</pre>
### prove this to yourself now

Copy/paste the code below into a new file in Komodo.  Save it to <code>/var/www/html/interfaces/cool.rb</code>

In your terminal window, cd into /var/www/html/interfaces/

Set the ruby script to be globally executable: <code> $  chmod a+x cool.rb  </code>

Then click on this link:  http://localhost/interfaces/cool.rb



In [5]:
#!/usr/bin/ruby

print "Content-Type: text/html  \n\n";

print "<html>
<head>
	<title>My first DYNAMIC web page</title>
</head>
<body>
	<h1>A dynamic Page</h1>
CGI programming is <b><i>cool!!</i></b>
<br/>
    
</body>
<html>  ";


Content-Type: text/html  

<html>
<head>
	<title>My first DYNAMIC web page</title>
</head>
<body>
	<h1>A dynamic Page</h1>
CGI programming is <b><i>cool!!</i></b>
<br/>
    
</body>
<html>  

<pre>
  

</pre>
  
# And there you are!  You are now a Web developer!

Congratulations! 
